# Using Azure OpenAI GPT models

This sample demonstrates how to get started with Azure OpenAI Chat Completions.

In [ ]:
%pip install openai --quiet
%pip install python-dotenv --quiet

In [ ]:
from dotenv import load_dotenv
import os

if os.path.exists(".env"):
    load_dotenv(override=True)

## Creating a chat completion service

>Note: The AzureChatCompletion service also supports Microsoft Entra authentication. If you don't provide an API key, the service will attempt to authenticate using the Entra token.

In [ ]:
from openai import AzureOpenAI

# Add Azure OpenAI chat completion
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

# Prepare the chat prompt
messages = [
    {
        "role": "system", "content": "You are an AI assistant that helps people find information.",
    },
    {
        "role": "user", "content": "What is the capital of France?"
    },
]

# Generate the completion
response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    messages=messages,
    max_tokens=800,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
    logprobs=True,
    top_logprobs=3
)

# Print the response from the model
print(response.choices[0].message.content)

Lets view the full JSON response.

In [ ]:
# Print the full response object
print(response.to_json())

Let's view the probabilities of each word.

In [ ]:
import numpy as np
from IPython.display import display, HTML

html_content = ""

for content in response.choices[0].logprobs.content:

    top_logprobs = content.top_logprobs

    for i, logprob in enumerate(top_logprobs, start=1):
        html_content += (
            f"<span style='color: cyan'>Output token {i}:</span> {logprob.token}, "
            f"<span style='color: darkorange'>logprobs:</span> {logprob.logprob}, "
            f"<span style='color: magenta'>linear probability:</span> {np.round(np.exp(logprob.logprob)*100,2)}%<br>"
        )

    html_content += f"<br>"
    
display(HTML(html_content))